In [4]:
import machine_learning as ml
import pandas as pd

In [5]:
train_a = pd.read_parquet('A/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')

In [6]:
y = train_a.dropna()
X_train_estimated_a = X_train_estimated_a.drop("date_calc", axis = 1)

X = pd.concat([X_train_observed_a, X_train_estimated_a], axis = 0, ignore_index=True)

In [7]:
ml.create_learner(X, y, X_test_estimated_a)

# Predict for B

There should be created a dataclass that takes all and predics :/

In [ ]:
train_b = pd.read_parquet('B/train_targets.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')

In [ ]:
y = train_b.dropna()
X_train_estimated_b = X_train_estimated_b.drop("date_calc", axis = 1)

X = pd.concat([X_train_observed_b, X_train_estimated_b], axis = 0, ignore_index=True)

In [ ]:
ml.create_learner(X, y, X_test_estimated_b)